In [1]:
#Effect Modification by Lifestyle Score
library(multcomp)
library(ggpubr)
library(dplyr)
library(lme4)
library(ggplot2)
library(rms)
library(mgcv)
library(tidymv)
library("ggthemes")
library(broom)
library(erer)
library(stringr)
library(tidyverse)
library(interactionR)
library(readr)
library(R.utils)
library(data.table)

#Read the complete set (no imputation)
UKBB_AG2 <- read.delim("~/jupyter/UKBB_AG2_07NOV20.txt", header = TRUE, na.strings=c("",".","NA"))


#Dichotomize Outcomes for Logistic Regression
#1_CKD
UKBB_AG2$CKD_only.0.0 <- factor(ifelse(UKBB_AG2$CKD.0.0=="CKD controls","CKD controls",
                              ifelse(UKBB_AG2$CKD.0.0=="CKD","CKD",NA)),
                levels = c("CKD controls", "CKD"))
#Set the refernece
UKBB_AG2$CKD_only.0.0 <- relevel(UKBB_AG2$CKD_only.0.0, ref = "CKD controls")

#2_CKD Extreme
UKBB_AG2$CKD_ex.0.0 <- factor(ifelse(UKBB_AG2$CKD.0.0=="CKD controls","CKD controls",
                              ifelse(UKBB_AG2$CKD.0.0=="CKD extreme","CKD extreme",NA)),
                levels = c("CKD controls", "CKD extreme"))
#Set the refernece
UKBB_AG2$CKD_ex.0.0 <- relevel(UKBB_AG2$CKD_ex.0.0, ref = "CKD controls")

#3_Micro
UKBB_AG2$micro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="micro","micro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "micro"))
#Set the reference
UKBB_AG2$micro.0.0 <- relevel(UKBB_AG2$micro.0.0, ref = "normo")

#4_Macro
UKBB_AG2$macro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "macro"))
#Set the reference
UKBB_AG2$macro.0.0 <- relevel(UKBB_AG2$macro.0.0, ref = "normo")

#5_Macro
UKBB_AG2$macro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "macro"))

#6_ESKD vs. Macro
UKBB_AG2$ESKD_macro.0.0 <- factor(ifelse(UKBB_AG2$ESKD.0.0=="yes","ESKD",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",NA)),
                levels = c("macro","ESKD"))

#7_DNCKD vs. Control DNCKD
UKBB_AG2$DNCKD2.0.0 <- factor(ifelse(UKBB_AG2$DNCKD.0.0=="yes","DNCKD",
                              ifelse(UKBB_AG2$CTRL_DNCKD.0.0=="yes","DNCKD Control",NA)),
                levels = c("DNCKD Control","DNCKD"))

#8_ESKD vs. Normo, Macro, Micro
UKBB_AG2$ESKD_Albu.0.0 <- factor(ifelse(UKBB_AG2$ESKD.0.0=="yes","ESKD",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0 %in% c("normo","macro","micro"),"albu",NA)),
                levels = c("albu","ESKD"))

#Set the reference
UKBB_AG2$macro.0.0 <- relevel(UKBB_AG2$macro.0.0, ref = "normo")

#Summarize Counts of Disease Outcomes
table(UKBB_AG2$CKD_only.0.0) #1
table(UKBB_AG2$CKD_ex.0.0) #2
table(UKBB_AG2$micro.0.0) #3
table(UKBB_AG2$macro.0.0) #4
table(UKBB_AG2$ESKD.0.0) #5
table(UKBB_AG2$DN.0.0) #6
table(UKBB_AG2$ALL.0.0) #7
table(UKBB_AG2$ESKD.0.0) #8
table(UKBB_AG2$ESKD_macro.0.0) #8
table(UKBB_AG2$ESKD_Albu.0.0) #9
table(UKBB_AG2$DNCKD2.0.0) #10

#table smoking
table(UKBB_AG2$'SMOKE.0.0')
UKBB_AG2 <- UKBB_AG2 %>% filter(SMOKE.0.0!='Prefer not to answer')
table(UKBB_AG2$'SMOKE.0.0')

Loading required package: mvtnorm

Warning message:
“package ‘mvtnorm’ was built under R version 3.6.3”
Loading required package: survival

Warning message:
“package ‘survival’ was built under R version 3.6.3”
Loading required package: TH.data

Loading required package: MASS

Warning message:
“package ‘MASS’ was built under R version 3.6.3”

Attaching package: ‘TH.data’


The following object is masked from ‘package:MASS’:

    geyser


Loading required package: ggplot2

Warning message:
“package ‘ggplot2’ was built under R version 3.6.3”

Attaching package: ‘dplyr’


The following object is masked from ‘package:MASS’:

    select


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“package ‘lme4’ was built under R version 3.6.3”
Loading required package: Matrix

Warning message:
“package ‘Matrix’ was built under R version 3.6.3”
Registered S3 method


CKD controls          CKD 
      349669         6108 


CKD controls  CKD extreme 
      349669          984 


 normo  micro 
348496  14070 


 normo  macro 
348496   1120 

...


macro  ESKD 
  963   447 


  albu   ESKD 
346560    447 


DNCKD Control         DNCKD 
       326513           645 


             Current                Never Prefer not to answer 
               38166               201909                 1269 
            Previous 
              132143 


             Current                Never Prefer not to answer 
               38166               201909                    0 
            Previous 
              132143 

In [2]:
#take apart package (interactionR), it breaks
interactionR_04Dec <- function(model, exposure_names = c(), ci.level = 0.95) {

          e1 <- grep(exposure_names[1], names(coef(model)), value = TRUE, ignore.case = TRUE)
          e2 <- grep(exposure_names[2], names(coef(model)), value = TRUE, ignore.case = TRUE)
          exposure_names <- union(e1, e2)
          beta1 <- exposure_names[1]
          beta2 <- exposure_names[3]
          beta3 <- exposure_names[2]

          b1 <- coef(M1)[beta1]
          b2 <- coef(model)[beta2]
          b3 <- coef(model)[beta3]

          se_vec <- summary(model)$coefficients[, 2] # extracts the SE vector for the coefficients
          # from the model
          v1 <- se_vec[beta1]^2
          v2 <- se_vec[beta2]^2
          v3 <- se_vec[beta3]^2

          ### Extracts the variance-covariance matrix from the model### for use in
          ### the delta and MOVER method CI estimation for RERI and AP###
          v_cov <- vcov(model)
          varNames <- c(beta1, beta2, beta3)
          v_cov1 <- v_cov[varNames, varNames] # for deltamethod
          cov12 <- v_cov[beta1, beta2]
          cov13 <- v_cov[beta1, beta3]
          cov23 <- v_cov[beta2, beta3]
          v123 <- v1 + v2 + v3 + (2 * (cov12 + cov13 + cov23))
          v12 <- v1 + v2 + (2 * (cov12))
          v13 <- v1 + v3 + (2 * (cov13))
          v23 <- v2 + v3 + (2 * (cov23))

          # Estimates individual and joint effects ORs (with CI) from the model

          alpha <- 1 - ci.level
          z <- qnorm(1 - alpha / 2)
    
          OR00 <- 1 # reference OR
          OR10 <- as.numeric(exp(b1))
          l1 <- exp(confint.default(model)[beta1, 1])
          u1 <- exp(confint.default(model)[beta1, 2]) # This is also OR of X on D (A==0)
          OR01 <- as.numeric(exp(b2))
          l2 <- exp(confint.default(model)[beta2, 1])
          u2 <- exp(confint.default(model)[beta2, 2]) # This is also OR of A on D (X==0)
          OR11 <- as.numeric(exp(b1 + b2 + b3))
          l3 <- exp(b1 + b2 + b3 - z * sqrt(v123))
          u3 <- exp(b1 + b2 + b3 + z * sqrt(v123))
    
    d <- data.frame(Measures = c("OR00", "OR01", "OR10", "OR11"), Estimates = c(
      OR00, OR01, OR10, OR11), CI.ll = c(NA,l2, l1, l3), 
                    CI.ul = c(NA, u2, u1, u3))
    rownames(d) <- NULL
  return(d)
}

In [3]:
#Review covariates
table(UKBB_AG2$LIFESCORE)
table(UKBB_AG2$GRS_WT_IR2d2)


     0      1      2 
 51042 136146 162002 


  high risk    low risk medium risk 
      37220       37281      297716 

In [4]:
#QA Check of Lifescore
#table(UKBB_AG2$LIFESCORE,UKBB_AG2$PACOUNT)
#good, checks out

#Dichotomous Dummy Variables
UKBB_AG2 <- UKBB_AG2 %>%
mutate(LIFESCORE_cat1=case_when(LIFESCORE==2 ~ 0,
                                LIFESCORE==1 ~ 1)) %>%
mutate(LIFESCORE_cat2=case_when(LIFESCORE==2 ~ 0,
                                LIFESCORE==0 ~ 1)) %>%

mutate(GRS_WT_IR2d2c=as.character(GRS_WT_IR2d2)) 

#Three deciles
UKBB_AG2 <- UKBB_AG2 %>%
mutate(WT_IR2d2_cat1=case_when(GRS_WT_IR2d2c=="low risk" ~ 0,
                               GRS_WT_IR2d2c=="high risk" ~ 1)) %>%
mutate(WT_IR2d2_cat2=case_when(GRS_WT_IR2d2c=="low risk" ~ 0,
                               GRS_WT_IR2d2c=="medium risk" ~ 1)) 

#Factor level coding
 UKBB_AG2$LIFESCORE_cat1 <- factor(UKBB_AG2$LIFESCORE_cat1, levels = c(0,1),
                 labels = c("Ref_Ideal","Intermediate"))
 UKBB_AG2$LIFESCORE_cat2 <- factor(UKBB_AG2$LIFESCORE_cat2, levels = c(0,1),
                 labels = c("Ref_Ideal","LowScore"))
 UKBB_AG2$WT_IR2d2_cat1 <- factor(UKBB_AG2$WT_IR2d2_cat1, levels = c(0,1),
                 labels = c("Ref_LowRisk","HighRisk"))
 UKBB_AG2$WT_IR2d2_cat2 <- factor(UKBB_AG2$WT_IR2d2_cat2, levels = c(0,1),
                 labels = c("Ref_LowRisk","MedRisk"))

#QA
#dim(UKBB_AG2)
table(UKBB_AG2$LIFESCORE_cat1)
table(UKBB_AG2$LIFESCORE_cat2)


   Ref_Ideal Intermediate 
      162002       136146 


Ref_Ideal  LowScore 
   162002     51042 

In [5]:
#Interaction Analysis

#Restriction: T2D, ND
T2D_UKBB <- UKBB_AG2 %>% filter(T2D_status==1) 
ND_UKBB <- UKBB_AG2 %>% filter(T2D_status==0) 

###MACRO###
#CKD - T2D, Low Lifestyle, High Risk
M1 <- glm(macro.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
#summary(M1)
value <- interactionR(M1,exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat2"),ci.type = "delta", ci.level = 0.95,
em = TRUE, recode = FALSE)
#names(value)
value$dframe
interactionR_table(value)

Warning message in log((exp(b1 + b2 + b3) - 1)):
“NaNs produced”
Warning message in log(.expr4):
“NaNs produced”


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,1.6802861,0.36331049,7.771208
OR10,1.3992951,0.42041594,4.657356
OR11,0.6373625,0.11434680,3.552622
"OR(relevel(LIFESCORE_cat2, ref = ""Ref_Ideal"")LowScore on outcome [relevel(WT_IR2d2_cat1, ref = ""Ref_LowRisk"")HighRisk==0]",1.6802861,0.36331049,7.771208
"OR(relevel(LIFESCORE_cat2, ref = ""Ref_Ideal"")LowScore on outcome [relevel(WT_IR2d2_cat1, ref = ""Ref_LowRisk"")HighRisk==1]",0.4554882,0.09619320,2.156801
Multiplicative scale,0.2710778,0.03086302,2.380946
RERI,-1.4422187,-4.80918183,1.924744


Do you want to save a Microsoft Word copy of the em/interaction table to your working directory? (yes/No/cancel) No
a flextable object.
col_keys: `*`, `relevel(LIFESCORE_cat2, ref = "Ref_Ideal")LowScore absent`, `relevel(LIFESCORE_cat2, ref = "Ref_Ideal")LowScore present`, `Effect of relevel(LIFESCORE_cat2, ref = "Ref_Ideal")LowScore within the strata of relevel(WT_IR2d2_cat1, ref = "Ref_LowRisk")HighRisk` 
header has 1 row(s) 
body has 6 row(s) 
original dataset sample: 
                                                            *
1                                                        <NA>
2                                                        <NA>
3  relevel(WT_IR2d2_cat1, ref = "Ref_LowRisk")HighRisk absent
4 relevel(WT_IR2d2_cat1, ref = "Ref_LowRisk")HighRisk present
5                                        Multiplicative scale
  relevel(LIFESCORE_cat2, ref = "Ref_Ideal")LowScore absent
1                                                      <NA>
2                              

In [6]:
#MACRO, T2D#

#01 - Low Score (Lifestyle), 10 - High (PGS)
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat2"), ci.level = 0.95)
#Macro, T2D, Intermediate Lifestyle, Intermiedate Risk
M1 <- glm(macro.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Intermediate Lifestyle, High Risk
M1 <- glm(macro.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Low Lifestyle, Moderate Risk
M1 <- glm(macro.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat2"), ci.level = 0.95)

Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,1.6802861,0.3633105,7.771208
OR10,1.3992951,0.4204159,4.657356
OR11,0.6373625,0.1143468,3.552622


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.4417049,0.08036617,2.427678
OR10,1.4429937,0.52076106,3.998438
OR11,1.6457152,0.59733649,4.534092


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.4677012,0.08452069,2.588057
OR10,1.4337504,0.43272929,4.750407
OR11,1.8794452,0.59678624,5.918893


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.000000,NA,NA
OR01,1.555662,0.3444958,7.025006
OR10,1.456027,0.5258270,4.031774
OR11,2.927120,1.0582131,8.096700


In [7]:
#MACRO, ND#

#01 - Low Score (Lifestyle), 10 - High (PGS)
M1 <- glm(macro.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat2"), ci.level = 0.95)
#Macro, T2D, Intermediate Lifestyle, Intermiedate Risk
M1 <- glm(macro.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Intermediate Lifestyle, High Risk
M1 <- glm(macro.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Low Lifestyle, Moderate Risk
M1 <- glm(macro.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat2"), ci.level = 0.95)

Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,1.1059197,0.5871916,2.082895
OR10,0.9383204,0.5921559,1.486847
OR11,1.0582448,0.5514559,2.030774


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.9773668,0.6077108,1.571876
OR10,0.7299413,0.5170071,1.030574
OR11,1.0176816,0.7232133,1.432048


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.9773820,0.6074654,1.572560
OR10,0.9473207,0.5980190,1.500649
OR11,1.5013101,0.9756952,2.310078


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,1.1083272,0.5895139,2.083732
OR10,0.7282957,0.5158272,1.028280
OR11,1.4054916,0.9771688,2.021561


In [8]:
#MICRO, T2D#

#01 - Low Score (Lifestyle), 10 - High (PGS)
M1 <- glm(micro.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat2"), ci.level = 0.95)
#Macro, T2D, Intermediate Lifestyle, Intermiedate Risk
M1 <- glm(micro.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Intermediate Lifestyle, High Risk
M1 <- glm(micro.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Low Lifestyle, Moderate Risk
M1 <- glm(micro.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat2"), ci.level = 0.95)

Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,1.7812593,1.1817092,2.684996
OR10,0.8120197,0.5640018,1.169103
OR11,1.4891120,1.0214039,2.170987


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,1.2416250,0.8673485,1.777409
OR10,0.9640161,0.7258184,1.280385
OR11,1.1752286,0.8869109,1.557273


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,1.2634524,0.8818944,1.810094
OR10,0.7866917,0.5467651,1.131901
OR11,1.1980778,0.8568647,1.675166


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,1.7777340,1.1834146,2.670525
OR10,0.9691337,0.7297556,1.287034
OR11,1.2858585,0.9613298,1.719943


In [9]:
#MICRO, ND#

#01 - Low Score (Lifestyle), 10 - High (PGS)
M1 <- glm(micro.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat2"), ci.level = 0.95)
#Macro, T2D, Intermediate Lifestyle, Intermiedate Risk
M1 <- glm(micro.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Intermediate Lifestyle, High Risk
M1 <- glm(micro.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Low Lifestyle, Moderate Risk
M1 <- glm(micro.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat2"), ci.level = 0.95)

Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.000000,NA,NA
OR01,1.252760,1.0477495,1.497884
OR10,1.073558,0.9425105,1.222826
OR11,1.400481,1.1770785,1.666284


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.000000,NA,NA
OR01,0.965864,0.8396554,1.111043
OR10,1.003728,0.9092410,1.108033
OR11,1.167912,1.0575918,1.289740


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.9679302,0.8412934,1.113629
OR10,1.0771442,0.9456948,1.226865
OR11,1.2237412,1.0708969,1.398400


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.000000,NA,NA
OR01,1.250764,1.0466636,1.494664
OR10,1.002712,0.9083019,1.106936
OR11,1.337017,1.2011107,1.488301


In [10]:
#CKD, T2D#

#01 - Low Score (Lifestyle), 10 - High (PGS)
M1 <- glm(CKD_only.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat2"), ci.level = 0.95)
#Macro, T2D, Intermediate Lifestyle, Intermiedate Risk
M1 <- glm(CKD_only.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Intermediate Lifestyle, High Risk
M1 <- glm(CKD_only.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Low Lifestyle, Moderate Risk
M1 <- glm(CKD_only.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat2"), ci.level = 0.95)

Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.7587211,0.3298465,1.745229
OR10,0.8306257,0.4689538,1.471230
OR11,0.8078115,0.4053347,1.609927


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.7638221,0.4054535,1.438942
OR10,0.8002021,0.5123547,1.249766
OR11,0.9396667,0.6033875,1.463361


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.7828771,0.4145889,1.478324
OR10,0.8082600,0.4573236,1.428495
OR11,0.9963108,0.5787801,1.715047


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.7586918,0.3326257,1.730513
OR10,0.8059308,0.5160645,1.258611
OR11,1.0376427,0.6543391,1.645481


In [11]:
#CKD, ND#

#01 - Low Score (Lifestyle), 10 - High (PGS)
M1 <- glm(CKD_only.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat2"), ci.level = 0.95)
#Macro, T2D, Intermediate Lifestyle, Intermiedate Risk
M1 <- glm(CKD_only.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Intermediate Lifestyle, High Risk
M1 <- glm(CKD_only.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Low Lifestyle, Moderate Risk
M1 <- glm(CKD_only.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat2"), ci.level = 0.95)

Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.000000,NA,NA
OR01,1.209364,0.9137047,1.600694
OR10,1.118955,0.9171318,1.365190
OR11,1.403050,1.0711257,1.837831


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.000000,NA,NA
OR01,1.326566,1.0854953,1.621175
OR10,1.097061,0.9424675,1.277012
OR11,1.248014,1.0711384,1.454097


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.000000,NA,NA
OR01,1.326473,1.0851183,1.621510
OR10,1.118758,0.9169963,1.364913
OR11,1.481638,1.2157540,1.805671


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.000000,NA,NA
OR01,1.203230,0.9099534,1.591030
OR10,1.096081,0.9416040,1.275901
OR11,1.608703,1.3671117,1.892987


In [12]:
#All, T2D#

#01 - Low Score (Lifestyle), 10 - High (PGS)
M1 <- glm(ALL.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat2"), ci.level = 0.95)
#Macro, T2D, Intermediate Lifestyle, Intermiedate Risk
M1 <- glm(ALL.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Intermediate Lifestyle, High Risk
M1 <- glm(ALL.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Low Lifestyle, Moderate Risk
M1 <- glm(ALL.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat2"), ci.level = 0.95)

Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,1.8652999,1.248622,2.786548
OR10,0.8976211,0.633049,1.272767
OR11,1.4663361,1.012822,2.122922


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.000000,NA,NA
OR01,1.242197,0.8731464,1.767235
OR10,1.019418,0.7737315,1.343117
OR11,1.250282,0.9510334,1.643692


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,1.2691441,0.8916761,1.806404
OR10,0.8710764,0.6147683,1.234244
OR11,1.2689960,0.9177461,1.754680


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.000000,NA,NA
OR01,1.845405,1.2391515,2.748267
OR10,1.025114,0.7782127,1.350349
OR11,1.423983,1.0745278,1.887088


In [13]:
#All, ND#

#01 - Low Score (Lifestyle), 10 - High (PGS)
M1 <- glm(ALL.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat2"), ci.level = 0.95)
#Macro, T2D, Intermediate Lifestyle, Intermiedate Risk
M1 <- glm(ALL.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Intermediate Lifestyle, High Risk
M1 <- glm(ALL.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Low Lifestyle, Moderate Risk
M1 <- glm(ALL.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat2"), ci.level = 0.95)

Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.000000,NA,NA
OR01,1.250271,1.0537606,1.483427
OR10,1.055406,0.9313531,1.195983
OR11,1.381765,1.1689179,1.633368


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.9657018,0.8448433,1.103850
OR10,0.9861316,0.8970815,1.084021
OR11,1.1630023,1.0576992,1.278789


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.9682063,0.8469045,1.106882
OR10,1.0590915,0.9346417,1.200112
OR11,1.2495912,1.1006919,1.418633


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,1.2466713,1.0512615,1.478404
OR10,0.9852283,0.8962403,1.083052
OR11,1.3463727,1.2152565,1.491635


In [14]:
#DN, T2D#

#01 - Low Score (Lifestyle), 10 - High (PGS)
M1 <- glm(DN.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat2"), ci.level = 0.95)
#Macro, T2D, Intermediate Lifestyle, Intermiedate Risk
M1 <- glm(DN.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Intermediate Lifestyle, High Risk
M1 <- glm(DN.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Low Lifestyle, Moderate Risk
M1 <- glm(DN.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=T2D_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat2"), ci.level = 0.95)

Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,1.5833571,0.3444921,7.277438
OR10,1.7973160,0.5624193,5.743659
OR11,0.9870003,0.2172143,4.484832


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.4458831,0.08122217,2.447752
OR10,1.6777296,0.60881070,4.623402
OR11,2.0187011,0.73708397,5.528751


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.4613921,0.08355443,2.547832
OR10,1.8175456,0.56894157,5.806347
OR11,2.0430151,0.65673591,6.355539


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.000000,NA,NA
OR01,1.438368,0.3189907,6.485773
OR10,1.694097,0.6149556,4.666944
OR11,3.409586,1.2395971,9.378271


In [15]:
#DN, ND#

#01 - Low Score (Lifestyle), 10 - High (PGS)
M1 <- glm(DN.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat2"), ci.level = 0.95)
#Macro, T2D, Intermediate Lifestyle, Intermiedate Risk
M1 <- glm(DN.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Intermediate Lifestyle, High Risk
M1 <- glm(DN.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat1","LIFESCORE_cat1"), ci.level = 0.95)
##Macro, T2D, Low Lifestyle, Moderate Risk
M1 <- glm(DN.0.0 ~ relevel(WT_IR2d2_cat2,ref="Ref_LowRisk")*relevel(LIFESCORE_cat2,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial) 
interactionR_04Dec(M1, exposure_names = c("WT_IR2d2_cat2","LIFESCORE_cat2"), ci.level = 0.95)

Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,1.1027059,0.6340208,1.917855
OR10,0.7742438,0.5072537,1.181763
OR11,1.1382599,0.6545668,1.979379


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.9243315,0.6075924,1.406187
OR10,0.7795237,0.5778532,1.051577
OR11,1.0830650,0.8051585,1.456893


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,0.9367511,0.6155349,1.425594
OR10,0.7746388,0.5076341,1.182082
OR11,1.4602747,1.0005236,2.131286


Measures,Estimates,CI.ll,CI.ul
<fct>,<dbl>,<dbl>,<dbl>
OR00,1.0000000,NA,NA
OR01,1.0808664,0.6224549,1.876879
OR10,0.7793127,0.5776889,1.051307
OR11,1.3665759,0.9950074,1.876900


In [16]:
#Significance Test#
#PGS modeled as 3 category; Lifestyle Score modeled as 4 category#

#Lifescore category
#Dichotomous Dummy Variables
table(ND_UKBB$LIFESCORE)
table(T2D_UKBB$LIFESCORE)

#DN-ND
#M1 <- glm(DN.0.0 ~ relevel(WT_IR2d2_cat1,ref="Ref_LowRisk")*relevel(LIFESCORE_cat1,ref="Ref_Ideal") + AGE.0.0 + SEX.0.0 + PC1 +
                                        #PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data=ND_UKBB, family=binomial)